In [56]:
import math 
import pickle
import plotly
import numpy as np
import pydaisi as pyd
from deepdiff import DeepDiff

shipment_delivery_optimization = pyd.Daisi("soul0101/Shipment Delivery Optimization")

[23:39:57] INFO     Calling https://app.daisi.io/pebble-api/daisies/connect?name=soul0101/Shipment   ]8;id=22686;file://c:\Users\91983\anaconda3\envs\face\lib\site-packages\pydaisi\pydaisi.py\pydaisi.py]8;;\:]8;id=508425;file://c:\Users\91983\anaconda3\envs\face\lib\site-packages\pydaisi\pydaisi.py#411\411]8;;\
                    Delivery Optimization                                                                          

[23:39:58] INFO     Found existing Daisi: 8ce3be65-1268-4c6b-96f3-b222567d3f17                       ]8;id=31427;file://c:\Users\91983\anaconda3\envs\face\lib\site-packages\pydaisi\pydaisi.py\pydaisi.py]8;;\:]8;id=804036;file://c:\Users\91983\anaconda3\envs\face\lib\site-packages\pydaisi\pydaisi.py#424\424]8;;\

In [57]:
# Load dummy data
depot_locations, drop_locations, depot_ids, drop_ids, depot_capacities = shipment_delivery_optimization.get_dummy_data().value

with open('./data/dummy_depot_locations.pkl', 'rb') as handle:
    gold_depot_locations = pickle.load(handle)
with open('./data/dummy_drop_locations.pkl', 'rb') as handle:
    gold_drop_locations = pickle.load(handle) 
with open('./data/dummy_depot_ids.pkl', 'rb') as handle:
    gold_depot_ids = pickle.load(handle)
with open('./data/dummy_drop_ids.pkl', 'rb') as handle:
    gold_drop_ids = pickle.load(handle)
with open('./data/dummy_depot_capacities.pkl', 'rb') as handle:
    gold_depot_capacities = pickle.load(handle)

assert np.array_equal(np.array(depot_locations), np.array(gold_depot_locations))
assert np.array_equal(np.array(drop_locations), np.array(gold_drop_locations))
assert np.array_equal(np.array(depot_ids), np.array(gold_depot_ids))
assert np.array_equal(np.array(drop_ids), np.array(gold_drop_ids))
assert np.array_equal(np.array(depot_capacities), np.array(gold_depot_capacities))

In [58]:
# Plot locations of Drops and Depots
before_fig = shipment_delivery_optimization.get_locations_plot_plotly(depot_locations, drop_locations, 
                                                        depot_ids=depot_ids, drop_ids=drop_ids, 
                                                        depot_capacities=depot_capacities).value
assert type(before_fig) == plotly.graph_objs._figure.Figure
before_fig.show()

In [59]:
# Get Drop allocations for each Depot
allocation_results = shipment_delivery_optimization.get_allocations(depot_locations, drop_locations, 
                                                        depot_ids, drop_ids, depot_capacities).value
with open('./result/allocation_results.pkl', 'rb') as handle:
    gold_allocation_results = pickle.load(handle)

assert DeepDiff(allocation_results, gold_allocation_results) == {}

In [60]:
allocation_fig = shipment_delivery_optimization.get_allocations_plot_plotly(allocation_results).value
assert type(allocation_fig) == plotly.graph_objs._figure.Figure
allocation_fig.show()

In [61]:
# Generating mock vehicle_capacities_list
num_vehicles = 5
vehicle_capacities_list = []
for depot_id, depot_info in allocation_results.items():
    num_drops = len(depot_info["drops"])
    num_vehicles = math.ceil(num_drops / 4)
    vehicle_capacities = num_vehicles * [5]
    vehicle_capacities_list.append(vehicle_capacities)

# Calculation Parameters
search_timeout = 10
sb_first_sol = "AUTOMATIC"
sb_local_mh = "AUTOMATIC"

In [62]:
# Parallel Computation
parallel_routing_results = shipment_delivery_optimization.run_parallel_route_solver(allocation_results, 
                                                                vehicle_capacities_list, 
                                                                search_timeout=search_timeout, 
                                                                first_sol_strategy=sb_first_sol, 
                                                                ls_metaheuristic=sb_local_mh).value

# Serial Computation
serial_routing_results = shipment_delivery_optimization.run_serial_route_solver(allocation_results, vehicle_capacities_list, search_timeout=search_timeout, first_sol_strategy=sb_first_sol, ls_metaheuristic=sb_local_mh).value

[23:40:48] INFO     [18:10:48] WARNING  Your Daisi Map Execution is still in          pydaisi.py:155 ]8;id=532144;file://c:\Users\91983\anaconda3\envs\face\lib\site-packages\pydaisi\pydaisi.py\pydaisi.py]8;;\:]8;id=390228;file://c:\Users\91983\anaconda3\envs\face\lib\site-packages\pydaisi\pydaisi.py#281\281]8;;\
                                        Progress! These are PARTIAL RESULTS.                                       
                                                                                                                   

[23:40:53] INFO     [18:10:53] WARNING  Your Daisi Map Execution is still in          pydaisi.py:155 ]8;id=62017;file://c:\Users\91983\anaconda3\envs\face\lib\site-packages\pydaisi\pydaisi.py\pydaisi.py]8;;\:]8;id=175102;file://c:\Users\91983\anaconda3\envs\face\lib\site-packages\pydaisi\pydaisi.py#281\281]8;;\
                                        Progress! These are PARTIAL RESULTS.                                       
                                                                                                                   

In [63]:
with open('./result/routing_results.pkl', 'rb') as handle:
    gold_routing_results = pickle.load(handle)

assert parallel_routing_results == gold_routing_results
assert serial_routing_results == gold_routing_results

In [64]:
# Generate result plots    
route_fig = None
for route in parallel_routing_results:
    route_fig = shipment_delivery_optimization.get_route_plot_plotly(depot_locations, 
                                                                drop_locations, route, 
                                                                fig=route_fig).value

assert type(route_fig) == plotly.graph_objs._figure.Figure
route_fig.show()